In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import warnings

# 모든 경고를 무시
warnings.filterwarnings("ignore")

In [6]:
pip install transformers datasets torch

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 71.7 MB/s eta 0:00:00a 0:00:01
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
train_df = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/ratings_train.txt", sep='\t')

In [3]:
test_df = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/ratings_test.txt", sep='\t' )

In [4]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [7]:
test_df = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/ratings_test.txt", sep='\t' )

In [3]:
import os

model_path = "./KcELECTRA-base/saved_model/"
print(os.path.exists(model_path))  # True여야 정상적으로 경로가 인식된 것


True


In [4]:
from sklearn.metrics import accuracy_score, classification_report


# base

In [5]:
import pandas as pd
import torch
from transformers import ElectraForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, classification_report

# 모델 경로 설정
model_path = "./KcELECTRA-base/saved_model/"

# 모델과 토크나이저 로드
model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # 평가 모드로 설정

# 결측값 제거 및 문자열 타입 변환
test_df = test_df.dropna(subset=['document'])
test_df['document'] = test_df['document'].astype(str)

# 감정 분석 함수 정의
def predict_sentiment(text):
    """
    주어진 텍스트의 감정을 예측합니다.
    긍정: 1, 부정: 0
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction  # 1 (긍정) 또는 0 (부정)

# test_df에서 document 컬럼의 감정을 예측
test_df['predicted_label'] = test_df['document'].apply(predict_sentiment)

# 모델 성능 평가
true_labels = test_df['label']
predicted_labels = test_df['predicted_label']

# 정확도 계산
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"모델 정확도: {accuracy * 100:.2f}%")

# 분류 보고서 출력
print("\n분류 보고서:")
print(classification_report(true_labels, predicted_labels, target_names=["부정", "긍정"]))


/home/user/miniforge3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-28 09:44:38.931232: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-28 09:44:42.243549: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2024-11-28 09:44:42.243824: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2024-11-28 09:44:42.247875: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2024-11-28 09:44:42.672752: I 

모델 정확도: 90.71%

분류 보고서:
              precision    recall  f1-score   support

          부정       0.91      0.91      0.91     24826
          긍정       0.91      0.91      0.91     25171

    accuracy                           0.91     49997
   macro avg       0.91      0.91      0.91     49997
weighted avg       0.91      0.91      0.91     49997



# KLUE-RoBERTa

In [9]:
import pandas as pd
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, classification_report

# 모델 경로 설정
model_path = "./KLUE-RoBERTa/saved_model/"  # klue-roberta 모델 경로

# 1. 모델과 토크나이저 로드
model = RobertaForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # 평가 모드로 설정

# 2. 결측값 제거 및 문자열 변환
# 'document' 컬럼의 결측값 제거
test_df2 = test_df2.dropna(subset=['document'])

# 'document' 컬럼을 문자열로 변환
test_df2['document'] = test_df2['document'].astype(str)

# 3. 감정 분석 함수 정의
def predict_sentiment(text):
    """
    주어진 텍스트의 감정을 예측합니다.
    긍정: 1, 부정: 0
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction  # 1 (긍정) 또는 0 (부정)

# 4. test_df에서 document 컬럼의 감정을 예측
test_df2['predicted_label'] = test_df2['document'].apply(predict_sentiment)

# 5. 모델 성능 평가 (정확도 계산)
true_labels = test_df2['label']
predicted_labels = test_df2['predicted_label']

# 정확도 계산
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"모델 정확도: {accuracy * 100:.2f}%")

# 세부 성능 지표 출력
print("\n분류 보고서:")
print(classification_report(true_labels, predicted_labels, target_names=["부정", "긍정"]))


모델 정확도: 91.10%

분류 보고서:
              precision    recall  f1-score   support

          부정       0.92      0.90      0.91     24826
          긍정       0.91      0.92      0.91     25171

    accuracy                           0.91     49997
   macro avg       0.91      0.91      0.91     49997
weighted avg       0.91      0.91      0.91     49997



# base2


In [11]:
import pandas as pd
import torch
from transformers import ElectraForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, classification_report

# 모델 경로 설정
model_path = "./KcELECTRA-base2/saved_model/"

# 모델과 토크나이저 로드
model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # 평가 모드로 설정

# 결측값 제거 및 문자열 타입 변환
test_df = test_df.dropna(subset=['document'])
test_df['document'] = test_df['document'].astype(str)

# 감정 분석 함수 정의
def predict_sentiment(text):
    """
    주어진 텍스트의 감정을 예측합니다.
    긍정: 1, 부정: 0
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction  # 1 (긍정) 또는 0 (부정)

# test_df에서 document 컬럼의 감정을 예측
test_df['predicted_label'] = test_df['document'].apply(predict_sentiment)

# 모델 성능 평가
true_labels = test_df['label']
predicted_labels = test_df['predicted_label']

# 정확도 계산
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"모델 정확도: {accuracy * 100:.2f}%")

# 분류 보고서 출력
print("\n분류 보고서:")
print(classification_report(true_labels, predicted_labels, target_names=["부정", "긍정"]))


모델 정확도: 90.28%

분류 보고서:
              precision    recall  f1-score   support

          부정       0.90      0.90      0.90     24826
          긍정       0.91      0.90      0.90     25171

    accuracy                           0.90     49997
   macro avg       0.90      0.90      0.90     49997
weighted avg       0.90      0.90      0.90     49997



# kobert

In [6]:
pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import torch
from transformers import BertForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, classification_report

# 1. KoBERT 모델 및 토크나이저 로드
model_name = "monologg/kobert"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 이진 분류
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()  # 평가 모드로 설정

# 2. 테스트 데이터 전처리
# 테스트 데이터 프레임 (test_df 예제)
test_df = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/ratings_test.txt", sep='\t' )
# 'document' 열에 텍스트 데이터가, 'label' 열에 실제 레이블이 있다고 가정

# 결측값 제거 및 문자열 변환
test_df = test_df.dropna(subset=['document'])
test_df['document'] = test_df['document'].astype(str)

# 3. 감정 분석 함수 정의
def predict_sentiment(text):
    """
    KoBERT를 사용하여 주어진 텍스트의 감정을 예측합니다.
    긍정: 1, 부정: 0
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction

# 4. test_df에서 'document' 컬럼을 기반으로 감정을 예측
test_df['predicted_label'] = test_df['document'].apply(predict_sentiment)

# 5. 모델 성능 평가
true_labels = test_df['label']  # 실제 레이블
predicted_labels = test_df['predicted_label']  # 예측 레이블

# 정확도 계산
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"모델 정확도: {accuracy * 100:.2f}%")

# 세부 성능 지표 출력
print("\n분류 보고서:")
print(classification_report(true_labels, predicted_labels, target_names=["부정", "긍정"]))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


모델 정확도: 50.93%

분류 보고서:
              precision    recall  f1-score   support

          부정       0.50      0.71      0.59     24826
          긍정       0.52      0.31      0.39     25171

    accuracy                           0.51     49997
   macro avg       0.51      0.51      0.49     49997
weighted avg       0.51      0.51      0.49     49997



# Hanbert

In [8]:
test_df = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/ratings_test.txt", sep='\t' )

In [9]:
import pandas as pd
import torch
from transformers import BertForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, classification_report

# 1. HanBERT 모델 및 토크나이저 로드
model_name = "hanbert"  # HanBERT 모델 이름 (Hugging Face에서 사용 가능한 이름)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 이진 분류
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()  # 평가 모드로 설정

# 2. 테스트 데이터 전처리
# 테스트 데이터 프레임 예제
# test_df = pd.read_csv("your_test_file.csv")  # 예제 데이터 파일 로드
# 'document' 열에 텍스트 데이터가, 'label' 열에 실제 레이블이 있다고 가정

# 결측값 제거 및 문자열 변환
test_df = test_df.dropna(subset=['document'])
test_df['document'] = test_df['document'].astype(str)

# 3. 감정 분석 함수 정의
def predict_sentiment(text):
    """
    HanBERT를 사용하여 주어진 텍스트의 감정을 예측합니다.
    긍정: 1, 부정: 0
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction  # 1 (긍정) 또는 0 (부정)

# 4. test_df에서 'document' 컬럼을 기반으로 감정을 예측
test_df['predicted_label'] = test_df['document'].apply(predict_sentiment)

# 5. 모델 성능 평가
true_labels = test_df['label']  # 실제 레이블
predicted_labels = test_df['predicted_label']  # 예측 레이블

# 정확도 계산
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"모델 정확도: {accuracy * 100:.2f}%")

# 세부 성능 지표 출력
print("\n분류 보고서:")
print(classification_report(true_labels, predicted_labels, target_names=["부정", "긍정"]))


OSError: hanbert is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`